In [1]:
import pandas as pd
import numpy as np
import qiime2 as q2
import biom
from qiime2 import Artifact, Metadata, Visualization
from qiime2.plugins.emperor.actions import biplot
from qiime2.plugins.qurro.actions import loading_plot
from qiime2.plugins.gemelli.actions import (joint_rpca, filter_ordination, feature_correlation_table)
from qiime2.plugins.longitudinal.actions import volatility
from qiime2.plugins.diversity.visualizers import beta_group_significance
from qiime2.plugins.feature_table.methods import filter_samples
from biom import load_table
# from gemelli.rpca import rpca
from qiime2.plugins.gemelli.actions import rpca

# plotting
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import os

plt.style.use('ggplot')
%matplotlib inline

In [2]:
# load in biplot, biomtable, taxonomy, and metadata
table = Artifact.import_data("FeatureTable[Frequency]", "pierce_autism_2024/data/wolr2/gg2/zebra30/196551_zebra_2022_8.qza")
metadata = Metadata.load('data/metadata/Metadata_and_Cumulative_TP.tsv')
taxonomy = Artifact.import_data('FeatureData[Taxonomy]', '/home/l1joseph/pierce_autism_2024/data/wolr2/gg2/196551_zebra.taxonomy.qza')


In [3]:
# perform RPCA
# ordination, distance = rpca(table, min_sample_count=500, max_iterations=15)
ordination, distance = rpca(table, min_sample_count=500)


In [7]:
if not os.path.exists(f'data/wolr2/gg2/zebra30/rpca'):
    os.makedirs(f'data/wolr2/gg2/zebra30/rpca')

biplot_viz = biplot(ordination, metadata, ignore_missing_samples=True, number_of_features=50)
biplot_viz.visualization.save('data/wolr2/gg2/zebra30/rpca/emp_biplot_50.qzv')

'data/wolr2/gg2/zebra30/rpca/emp_biplot_50.qza.qzv'

In [14]:

ordination.save('data/wolr2/gg2/zebra30/rpca/ordination.qza')

'data/wolr2/gg2/zebra30/rpca/ordination.qza'

In [12]:
type(biplot_viz)


qiime2.sdk.results.Results

In [13]:
type(ordination)

qiime2.sdk.result.Artifact

In [12]:
# Ensure the output directory exists
if not os.path.exists(f'data/wolr2/gg2/zebra30/qurro'):
    os.makedirs(f'data/wolr2/gg2/zebra30/qurro')


# run Qurro metaG
qurro_plot_metag = loading_plot(ordination, table, metadata, feature_metadata=taxonomy.view(Metadata))
qurro_plot_metag.visualization.save(f'data/wolr2/gg2/zebra30/qurro/qurro-metag.qzv')

'data/wolr2/gg2/zebra30/qurro/qurro-metag.qzv'

## Empress Plot

In [10]:
!empress community-plot \
    --tree /projects/wol/qiyun/GG2/phylogeny/tree.nwk \
    --table /home/l1joseph/pierce_autism_2024/data/wolr2/gg2/zebra30/196551_zebra.biom/feature-table.biom \
    --sample-metadata data/metadata/Metadata_Final.tsv \
    --feature-metadata ref/lineages.txt \
    --output-dir data/wolr2/gg2/zebra30/empress/empress-tree.qzv

Usage: qiime empress community-plot [OPTIONS]

  Generates an interactive phylogenetic tree visualization supporting
  interaction with sample and feature metadata and, optionally, Emperor
  integration.

Inputs:
  --i-tree ARTIFACT    The phylogenetic tree to visualize.
    Phylogeny[Rooted]                                               [required]
  --i-feature-table ARTIFACT FeatureTable[Frequency¹ | RelativeFrequency² |
    PresenceAbsence³]  A table that documents which features are in which
                       samples. This information allows us to decorate the
                       phylogeny by sample metadata. It's expected that all
                       features in the table are also present as tips in the
                       tree, and that all samples in the table are also
                       present in the sample metadata file.         [required]
  --i-pcoa ARTIFACT    Principal coordinates matrix to display simultaneously
    PCoAResults        with the phylogenet

In [15]:
!qiime empress community-plot \
    --i-tree /projects/wol/qiyun/GG2/phylogeny/tree.nwk \
    --i-feature-table data/wolr2/gg2/zebra30/196551_zebra_2022_8.qza \
    --m-sample-metadata-file data/metadata/Metadata_Final.tsv \
    --m-feature-metadata-file ref/lineages.txt \
    --i-pcoa data/wolr2/gg2/zebra30/rpca/ordination.qza \
    --p-filter-extra-samples \
    --o-visualization data/wolr2/gg2/zebra30/empress/empire_q.qzv

Usage: qiime empress community-plot [OPTIONS]

  Generates an interactive phylogenetic tree visualization supporting
  interaction with sample and feature metadata and, optionally, Emperor
  integration.

Inputs:
  --i-tree ARTIFACT    The phylogenetic tree to visualize.
    Phylogeny[Rooted]                                               [required]
  --i-feature-table ARTIFACT FeatureTable[Frequency¹ | RelativeFrequency² |
    PresenceAbsence³]  A table that documents which features are in which
                       samples. This information allows us to decorate the
                       phylogeny by sample metadata. It's expected that all
                       features in the table are also present as tips in the
                       tree, and that all samples in the table are also
                       present in the sample metadata file.         [required]
  --i-pcoa ARTIFACT    Principal coordinates matrix to display simultaneously
    PCoAResults        with the phylogenet

In [5]:
!empress -h

Usage: empress [OPTIONS] COMMAND [ARGS]...
Try 'empress --help' for help.

Error: No such option: -h


creating gg2 taxonomy

In [3]:
!pip install q2-greengenes2

  Preparing metadata (setup.py) ... done
  Created wheel for q2-greengenes2: filename=q2_greengenes2-2024.1-py3-none-any.whl size=20592 sha256=0e3332e4dbae62e817648d6160f1ff866bf42c00f79114e789149d3a99e171b1
  Stored in directory: /home/l1joseph/.cache/pip/wheels/a3/99/9e/bc7588ae8946cd3dc33405fe53d8699ae03c261c9a4f994054
Successfully built q2-greengenes2


In [10]:
qiime greengenes2 filter-features \
     --i-feature-table woltka.example.biom.qza \
     --i-reference 2022.10.taxonomy.asv.nwk.qza \
     --o-filtered-feature-table woltka_gg2.example.biom.qza


NameError: name 'pd' is not defined

In [5]:
!qiime greengenes2 taxonomy-from-table \
     --i-reference-taxonomy /home/mcdonadt/greengenes2/release/2022.10/2022.10.taxonomy.asv.nwk.qza \
     --i-table /home/l1joseph/pierce_autism_2024/data/wolr2/gg2/zebra30/196551_zebra_2022_8.qza \
     --o-classification /home/l1joseph/pierce_autism_2024/data/wolr2/gg2/196551_zebra.taxonomy.qza

Saved FeatureData[Taxonomy] to: /home/l1joseph/pierce_autism_2024/data/wolr2/gg2/196551_zebra.taxonomy.qza
